In [20]:
pip install docx2python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [docx2python]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import docx
from typing import Dict, List, Tuple
import xml.etree.ElementTree as ET

def extract_with_equations(file_path: str) -> Dict[str, Dict[str, List[str]]]:
    """
    Extract questions and answers while preserving equations and chemical formulas
    """
    try:
        doc = docx.Document(file_path)
    except Exception as e:
        print(f"Error reading file: {e}")
        return {}
    
    
    result = {
        'PHYSICS': {'questions': [], 'answers': []},
        'CHEMISTRY': {'questions': [], 'answers': []},
        'MATHS': {'questions': [], 'answers': []}
    }
    
    current_subject = None
    current_question = []
    collecting_question = False
    question_number = 0
    
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        
        
        if not text:
            continue
            
        
        if text == 'PHYSICS':
            current_subject = 'PHYSICS'
            question_number = 0
            continue
        elif text == 'CHEMISTRY':
            current_subject = 'CHEMISTRY'
            question_number = 0
            continue
        elif text == 'MATHS':
            current_subject = 'MATHS'
            question_number = 0
            continue
        
        
        if text in ['SECTION - I', 'Single Correct Answer Type', 'SECTION - II', 'Numerical Type']:
            continue
        
       
        if re.match(r'^Q\.\d+', text) and current_subject:
            
            if current_question and collecting_question:
                question_text = '\n'.join(current_question)
                result[current_subject]['questions'].append(question_text)
                current_question = []
            
            current_question.append(text)
            collecting_question = True
            question_number += 1
            continue
        
        
        if ('[Solution :' in text or 'Solution :' in text) and current_subject and collecting_question:
            
            answer_match = re.search(r'[\(\[ ]([A-D1-9])[\)\]]', text)
            answer = answer_match.group(1) if answer_match else '?'
            result[current_subject]['answers'].append(answer)
            
            
            if current_question:
                question_text = '\n'.join(current_question)
                result[current_subject]['questions'].append(question_text)
                current_question = []
                collecting_question = False
            continue
        
        
        if collecting_question and current_subject:
            current_question.append(text)
    
    
    if current_question and collecting_question and current_subject:
        question_text = '\n'.join(current_question)
        result[current_subject]['questions'].append(question_text)
    
    return result

def extract_questions_answers_robust(file_path: str) -> Dict[str, Dict[str, List[str]]]:
    """
    More robust extraction that preserves all content including equations
    """
    try:
        doc = docx.Document(file_path)
    except Exception as e:
        print(f"Error reading file: {e}")
        return {}
    
    result = {
        'PHYSICS': {'questions': [], 'answers': []},
        'CHEMISTRY': {'questions': [], 'answers': []},
        'MATHS': {'questions': [], 'answers': []}
    }
    
    current_subject = None
    buffer = []
    in_question = False
    current_q_num = 0
    
    
    all_lines = []
    for para in doc.paragraphs:
        if para.text.strip():
            all_lines.append(para.text.strip())
    
    i = 0
    while i < len(all_lines):
        line = all_lines[i]
        
        
        if line in ['PHYSICS', 'CHEMISTRY', 'MATHS']:
            current_subject = line
            i += 1
            continue
        
        
        if any(header in line for header in ['SECTION -', 'Single Correct', 'Numerical Type']):
            i += 1
            continue
        
        
        if re.match(r'^Q\.\d+', line) and current_subject:
            if buffer and in_question:
                
                result[current_subject]['questions'].append('\n'.join(buffer))
                buffer = []
            
            buffer.append(line)
            in_question = True
            current_q_num += 1
            
           
            j = i + 1
            answer_found = False
            while j < len(all_lines) and j < i + 8:
                if '[Solution :' in all_lines[j]:
                    match = re.search(r'[\(\[ ]([A-D1-9])[\)\]]', all_lines[j])
                    if match:
                        result[current_subject]['answers'].append(match.group(1))
                        answer_found = True
                    break
                elif re.match(r'^Q\.\d+', all_lines[j]):
                    break
                else:
                    buffer.append(all_lines[j])
                j += 1
            
            if not answer_found:
                result[current_subject]['answers'].append('?')
            
            if buffer:
                result[current_subject]['questions'].append('\n'.join(buffer))
                buffer = []
            
            i = j
            in_question = False
        else:
            i += 1
    
    return result

def get_separate_arrays(results: Dict[str, Dict[str, List[str]]]) -> Dict[str, Tuple[List[str], List[str]]]:
    """
    Return separate arrays for questions and answers for each subject
    """
    separated = {}
    for subject, data in results.items():
        separated[subject] = (data['questions'], data['answers'])
    return separated

def display_separate_arrays(separated: Dict[str, Tuple[List[str], List[str]]]):
    """
    Display questions and answers as separate arrays
    """
    for subject, (questions, answers) in separated.items():
        print(f"\n{'='*80}")
        print(f"{subject}")
        print(f"{'='*80}")
        
        print(f"\nQUESTIONS ARRAY ({len(questions)} questions):")
        print("[")
        for i, q in enumerate(questions):
            print(f"  {i+1}: {q[:100]}..." if len(q) > 100 else f"  {i+1}: {q}")
        print("]")
        
        print(f"\nANSWERS ARRAY ({len(answers)} answers):")
        print(f"{answers}")
        
        print(f"\nMATCHING: {len(questions)} questions, {len(answers)} answers")

def save_separate_arrays(separated: Dict[str, Tuple[List[str], List[str]]], filename_prefix: str):
    """
    Save separate arrays to files
    """
    for subject, (questions, answers) in separated.items():
        
        q_filename = f"{filename_prefix}_{subject.lower()}_questions.txt"
        with open(q_filename, 'w', encoding='utf-8') as f:
            f.write(f"{subject} - QUESTIONS\n")
            f.write("="*60 + "\n\n")
            for i, q in enumerate(questions):
                f.write(f"Q.{i+1}:\n{q}\n")
                f.write("-"*40 + "\n\n")
        
        
        a_filename = f"{filename_prefix}_{subject.lower()}_answers.txt"
        with open(a_filename, 'w', encoding='utf-8') as f:
            f.write(f"{subject} - ANSWERS\n")
            f.write("="*60 + "\n\n")
            for i, a in enumerate(answers):
                f.write(f"Q.{i+1}: {a}\n")
        
        print(f"Saved {subject}: {q_filename}, {a_filename}")

def main():
    print("BTEST Question-Answer Extractor with Equations")
    print("="*50)
    
    file_path = 'SET - A.docx'
    
    if not file_path.endswith('.docx'):
        print("Please provide a .docx file")
        return
    
    try:
        
        print("Extracting questions and answers...")
        results = extract_questions_answers_robust(file_path)
        
       
        separated = get_separate_arrays(results)
        
        # Display results
        display_separate_arrays(separated)
        
        # Show summary
        total_questions = sum(len(questions) for questions, answers in separated.values())
        total_answers = sum(len(answers) for questions, answers in separated.values())
        print(f"\n{'='*50}")
        print(f"SUMMARY: {total_questions} questions, {total_answers} answers total")
        
        # Save option
        save = input("\nSave to files? (y/n): ").lower()
        if save == 'y':
            prefix = input("Enter filename prefix: ").strip() or "btest"
            save_separate_arrays(separated, prefix)
            print("Files saved successfully!")
    
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    # Uncomment to test with sample data first
    # test_with_sample()
    
    main()

BTEST Question-Answer Extractor with Equations
Extracting questions and answers...

PHYSICS

QUESTIONS ARRAY (74 questions):
[
  1: Q.1	A Capacitor stores  charge when connected across a battery. When the gap between the plates is f...
  2: Q.2	A capacitor is connected to a battery. The force of attraction between the plates when the separ...
  3: Q.3	A flat air capacitor  consists of two large plates that are close to each other. Initially, one ...
  4: Q.4	In the circuit shown, after the switch is shifted to position 2 the heat generated in  resistanc...
  5: Q.5	Consider the loop PQRSP, carrying clockwise current , shown in the figure. The magnitude of 	mag...
  6: Q.6	The straight long conductors AOB and COD are perpendicular to each other and carry current  and ...
  7: Q.7	A wire of length L carrying current I is bent into a circle of one turn. The field at the center...
  8: Q.8	A particle of mass  and charge  is in an electric and magnetic field given by
The charged partic...
 

In [ ]:
✅ Successfully formatted the questions by subject!

{'CHEMISTRY': ['Q.2 Point out the correct statements amongst the following (A) '
               'has tetrahedral geometry and hybridization (B) is octahedral '
               'and Ni has hybridization (C) is tetrahedral and diamagnetic '
               '(D) has octahedral geometry and hybridization',
               'Q.3 Ethyl iodide and n-propyl iodide are allowed to undergo '
               'Wurtz reaction. The alkane which will not be obtained in this '
               'reaction is: (A) Butane (B) Propane (C) Pentane (D) Hexane',
               'Q.4 The IUPAC name of compound Y is: (A) 2-(Cyclohexyl) butane '
               '(B) 1-(Methyl propyl) cyclohexane (C) Butyl cyclohexane (D) '
               '1-(Cyclohexyl) butane',
               'Q.5 If is major product then which of the following is false. '
               '(A) When B is treated with then an optically inactive product '
               'is formed. (B) When B is treated with HBr then an optically '
               'inactive mixture is formed. (C) When B is treated with then '
               'only one type of carbonyl compound will form. (D) When B is '
               'treated with THF followed by then aldehyde will form.',
               'Q.6 The major product is',
               'Q.7 Which of the following statement is not true for the '
               'reaction given below? (A) It is a ligand substitution reaction '
               '(B) is a relatively strong field ligand while is a weak field '
               'ligand (C) During the reaction, there is a change in colour '
               'from light blue to dark blue (D) , has a tetrahedral structure '
               'and is paramagnetic',
               'Q.8 Observe the following reactions The major product formed '
               'is (A) 1-methylcyclohexanol (B) - cis - 2-methylcyclohexanol '
               '(C) - trans - 2-methylcyclohexanol (D) ( )-2- '
               'methylcyclohexane',
               'Q.9 Group 14 elements can form halides of formulae and where '
               'and I. Except carbon, all other members react directly with '
               'halogen under suitable conditions to make halides. Identify '
               'the false statement (A) most of the are covalent in nature '
               'with hydridization at the central atom (B) Thermally and '
               'chemically is more stable than (C) and are more ionic in '
               'nature (D) is more stable than',
               'Q.10 Anhydrous is covalent however when it is dissolved in '
               'water hydrated ionic species are formed. This transformation '
               'is owing to: (A) The trivalent state of Al . (B) The large '
               'hydration energy of (C) The low hydration energy of (D) The '
               'Polar nature of water',
               'Q.11 Select the correct option: (A) Q can exhibit geometrical '
               'isomerism (B) Conversion from P to Q involves three transition '
               'states (C) Q gives two stereoisomers with cold dil. (D) '
               'Reductive ozonolysis of Q gives cyclic diketone',
               'Q.12 of gas at STP is adsorbed per gm of silica gel the area '
               'occupied by molecule is what is surface area per gram of '
               'silica gel (A) (B) (C) (D)',
               'Q.13 The number of isomers possible for is (A) 1 (B) 2 (C) 3 '
               '(D) 4',
               "Q.14 The hybridization of compound ' ' is: (A) (B) (C) (D)",
               'Q.15 The total number of possible structural isomers for the '
               'complex compound are (A) 3 (B) 4 (C) 5 (D) 6',
               'Q.16 On adding solution into KI solution, a negatively charged '
               'colloidal sol is obtained when they are in (A) 50 ml of of 0.1 '
               'M KI (B) 50 ml of of 0.2 M KI (C) 50 ml of of 0.1 M KI (D) All '
               'of these',
               'Q.17 If we arrange the following in order of magnitude of work '
               'done by the system/on the system at constant temperature then '
               'the one which will be highest is (A) for expansion in infinite '
               'stages. (B) for expansion in single stage. (C) for compression '
               'in infinite stages.. (D) for compression in single stage.',
               "Q.18 In which of the following cases 'd' orbital without any "
               'nodal plane is used in hybridization of central atom/ion? (A) '
               '(B) (C) (D)',
               'Q.19 Consider the following complexes. (P) (Q) Select the '
               'correct options. (A) Both P and Q have longer bond length than '
               'free CO molecule. (B) P has shorter CO bond length than Q . '
               '(C) P has longer bond length than Q (D)',
               'Q.20 It is an experimental fact that: salt Red precipitate '
               'Which of the following is wrong about this red precipitate? '
               '(A) It is a non-ionic complex. (B) It involves intra molecular '
               'H -bonding. (C) Ni (II) is hybridized. (D) It is a diamagnetic '
               'complex.',
               'Q.21 How many of the following when treated with water or '
               'dilute acid in cold condition give i) ii) iii) iv) v) vi) vii) '
               'viii)',
               'Q.22 Two complexes of chromium in have following formula and '
               'Where is a monodentate neutral ligand and is anionic bidentate '
               'ligand carrying -2 charge. If ratio of spin only magnetic '
               'moments of these two complexes is then Calculate',
               "Q.23 If' is total number of all possible compounds (including "
               'stereo isomers) which can show both geometrical and optical '
               "isomerism, then find the value of ' ' ?",
               'Q.24 Two cylinders “A” and “B” fitted with pistons contains '
               'equal amounts of an Ideal gas (diatomic) at 300K, the piston '
               'of “A” is free to move; while that “B” is held fixed. The same '
               'amount of heat is given to the gas in each Cylinder. If the '
               'rise in temperature of the gas is “A” is 30K. The rise in '
               'temperature of the gas in “B” is 7x what is the value of “x”',
               'Q.25 How many geometrical isomers are possible for complex'],
 'MATHS': ['Q.1 If y = f (x) makes +ve intercept of 2 and 0 unit x and y axes '
           'and encloses an area of 3/4 square unit with the axes then is (A) '
           '3/2 (B) 1 (C) 5/4 (D) -3/4',
           'Q.2 is (A) 20 (B) 8 (C) 10 (D) 18',
           'Q.3 Let for and . Then is equal to: (A) (B) 1 (C) (D) 2',
           'Q.4 If then is equal to (A) (B) (C) (D)',
           'Q.5 Let f (x) be a function satisfying f’ (x) = f (x) with f (0) = '
           '1 and (x) be a function that satisfies f (x) + (x) = x2. Then the '
           'value of the integral (A) (B) (C) (D)',
           'Q.6 is (A) (B) (C) (D)',
           'Q.7 The value of the integral I = (A) (B) (C) (D)',
           'Q.8 The value of is (A) 0 (B) 3 (C) 2 (D) 1',
           'Q.9 If , then is equal to: (A) 9 (B) (C) (D)',
           'Q.10 If a curve passes through the point and the area bounded by '
           'the curve, line and axis is 8 sq units, then (A) (B) (C) (D)',
           'Q.11 The area (in sq. units) of the region is (A) 30 (B) (C) 16 '
           '(D) 18',
           'Q.12 The area (in sq. units) bounded by the curves and the line is '
           '(A) (B) (C) (D)',
           'Q.13 Let be a continuous function such that for all Let , and be '
           'the area of the region bounded by , and the axis. Then (A) (B) (C) '
           '(D)',
           'Q.14 Circle centered at origin and having radius units is divided '
           'by the curve in two parts. Then area of upper parts equals to (A) '
           '(B) (C) (D)',
           'Q.15 The area bounded by the curves and is (A) (B) (C) (D)',
           'Q.16 For , let and be one of its root. Then, among the two '
           'statements (I) If , then cannot be the geometric mean of and (II) '
           'If , then may be the geometric mean of and (A) only (II) is true '
           '(B) Both (I) and (II) are true (C) only (I) is true (D) Neither '
           '(I) nor (II) is true',
           'Q.17 Let . If , then the sum of all elements of the matrix is (A) '
           '1 (B) 3 (C) 4 (D) 5',
           'Q.18 Let . If M and N are two matrices given by and then is: (A) a '
           'non-identity symmetric matrix (B) a skew-symmetric matrix (C) '
           'neither symmetric nor skew-symmetric matrix (D) an identity matrix',
           'Q.19 Let be 10 observations such that and . Then the standard '
           'deviation of is equal to: (A) 5 (B) (C) 10 (D)',
           'Q.20 Let and be two relation defined as follows : and , where Q is '
           'the set of all rational numbers. Then: (A) Neither nor is '
           'transitive. (B) is transitive but is not transitive. (C) and are '
           'both transitive. (D) is transitive but is not transitive.',
           'Q.21 Let denote the greatest integer . Then the value of 8. is .',
           "Q.22 Different A.P.'s are constructed with the first term 100, the "
           'last term 199, and integral common differences. The sum of the '
           "common differences of all such A.P.'s having at least 3 terms and "
           'at most 33 terms is .',
           'Q.23 The graphs of sine and cosine functions, intersect each other '
           'at a number of points and between two consecutive points of '
           'intersection, the two graphs enclose the same area . Then is equal '
           'to .',
           'Q.24 The sum of distinct values of for which the system of '
           'equations has non-zero solutions, is .',
           'Q.25 If the mean and variance of eight numbers and be 10 and 25 '
           'respectively, then is equal to _______.'],
 'PHYSICS': ['Q.1 A Capacitor stores charge when connected across a battery. '
             'When the gap between the plates is filled with a dielectric, a '
             'charge of flows through the battery, the dielectric constant of '
             'the material inserted is (A) 5 (B) 2 (C) 3 (D) 4',
             'Q.2 A capacitor is connected to a battery. The force of '
             'attraction between the plates when the separation between them '
             'is halved is (A) Remains the same (B) Becomes two times (C) '
             'Becomes eight times (D) Becomes four times',
             'Q.3 A flat air capacitor consists of two large plates that are '
             'close to each other. Initially, one of the plates was not '
             'charged, while the other had charge . If entire space between '
             'the plates is filled with a slab of finite electrical '
             'resistance, estimate total amount of energy lost in the slab. '
             '(A) (B) (C) (D)',
             'Q.4 In the circuit shown, after the switch is shifted to '
             'position 2 the heat generated in resistance is 7.5 J. Find the '
             'EMF of the cell. (A) 200 V (B) 460 V (C) 400 V (D) 180 V',
             'Q.5 Consider the loop PQRSP, carrying clockwise current , shown '
             'in the figure. The magnitude of magnetic field at the centre O '
             'of the curved portion is (A) (B) (C) 0 (D)',
             'Q.6 The straight long conductors AOB and COD are perpendicular '
             'to each other and carry current and . The magnitude of the '
             'magnetic induction at point P at a distance a from the point O '
             'in a direction perpendicular to the plane ACBD is (A) (B) (C) '
             '(D)',
             'Q.7 A wire of length L carrying current I is bent into a circle '
             'of one turn. The field at the center of the coil is . A similar '
             'wire of length carrying current is bent into a square of one '
             'turn. The field at its center is . Then (A) (B) (C) (D)',
             'Q.8 A particle of mass and charge is in an electric and magnetic '
             'field given by The charged particle shifted from the origin to '
             'the point along a straight line path. The magnitude of total '
             'work done is. (A) (B) 0.25q (C) (D)',
             'Q.9 Find the magnitude of the magnetic induction B of a magnetic '
             'field generated by a system of thin conductors along which a '
             'current i is flowing at a point , that is the centre of a '
             'circular conductor of radius . The ring is in plane. (A) (B) (C) '
             '(D) None of these',
             'Q.10 For if, the magnetic field at point will be zero when (A) '
             '(B) (C) (D)',
             'Q.11 Find the magnitude and direction of magnetic field at point '
             'due to the current carrying wire as shown in figure. (A) (B) (C) '
             '(D)',
             'Q.12 A particle of mass and charge is thrown from origin at with '
             'velocity units in a region with uniform magnetic field units. '
             'After time , and electric field is switched on such that '
             'particle moves on a straight line with constant speed. May be: '
             '(A) units (B) units (C) units (D) units',
             'Q.13 Figure shows a vertical force F that is applied '
             'tangentially to a uniform cylinder of weight . The coefficient '
             'of static friction between the cylinder and all surfaces is 0.5. '
             'Find in terms of W, the maximum force that can be applied '
             'without causing the cylinder to rotate. (A) (B) (C) (D)',
             'Q.14 A Particle having mass 2 KG is moving along straight line '
             'with speed . Find angular momentum of the particle about origin. '
             'and are in metres. (A) (B) (C) (D)',
             'Q.15 Figure shows a rough track, a portion of which is in from '
             'of a cylinder of radius . With what minimum linear speed should '
             'a sphere of radius be set rolling on the horizontal part so that '
             'it completely goes round the circle on the cylindrical part. (A) '
             '(B) (C) (D)',
             'Q.16 A cylinder rolls up an inclined plane, reaches some height '
             'and then rolls down (without slipping throughout these motions). '
             'The directions of the frictional force acting on the cylinder '
             'are: (A) Up the incline while ascending and down the incline '
             'while descending (B) Up the incline while ascending as well as '
             'descending (C) Down the incline while ascending and up the '
             'incline while descending (D) Down the incline while ascending as '
             'well as descending.',
             'Q.17 Water flows out of a big tank along a tube bent at right '
             'angles; the inside radius of the tube is equal to . The length '
             'of the horizontal section of the tube is equal to . The water '
             'flow rate is . Find the moment of reaction forces of flowing '
             "water, acting on the tube's walls, relative to the point O. "
             'density of water (A) (B) (C) (D) None of these.',
             "Q.18 A vessel with liquid of density ' ' falls with acceleration "
             ". Find the pressure in the vessel as a function of depth ' '. "
             'Given atmospheric pressure. (A) (B) (C) (D)',
             'Q.19 A piece of ice floats in a vessel with water above which a '
             'layer of lighter oil is poured. What happens to the level of oil '
             'and level of water in the vessel when the ice completely melts. '
             '(A) level of both oil and water increases. (B) level of oil '
             'increases and level of water remains same. (C) level of oil '
             'decreases and level of water increases. (D) level of water '
             'increases but level of oil remains same.',
             'Q.20 Incompressible fluid of negligible viscosity is pumped at a '
             'volume flow rate Q through a small hole into the narrow gap '
             'between two clearly spaced parallel plates as shown. The liquid '
             'flowing away from the hole only has radial motion. Find the '
             'speed as a function of radius from the hole. Separation between '
             "the plates is ' '. (A) (B) (C) (D)",
             'Q.21 A parallel plate capacitor which is disconnected is at a '
             'certain potential difference. When a 3 mm thick dielectric slab '
             'is introduced between the plates, the plate separation has to be '
             'increased by 2 mm in order to maintain the same potential '
             'difference between the plates. Find the dielectric constant of '
             'the slab.',
             'Q.22 A slender rod of length forming an angle theta with the '
             'horizontal strikes, a friction less floor at A with its centre '
             'of mass velocity Vc and no angular velocity. Assumed that the '
             'impact at A is perfectly inelastic, find the angular velocity of '
             'the rod immediately after the impact. Given :',
             'Q.23 Find the magnetic field due to current i flowing in an '
             "elliptical loop at it's focus. The equation of ellipse (in polar "
             'coordinates as shown) is . Here e is eccentricity which is a '
             'constant. Take , if your answer is , fill n in OMR sheet.',
             'Q.24 A uniform conducting wire has a mass of 10 g. A current of '
             '2A flows through it. The wire is kept in a uniform magnetic '
             "field . The acceleration of the wire is . Find ' '?",
             'Q.25 Two parallel long wires carry currents . When the currents '
             'are in the same direction, the magnetic field midway between the '
             'wires is . When the direction of is reversed, it becomes . If '
             "the ratio where ' ' and ' ' smallest integers, find ."]}